# P5 

This project gives you experience with getting data from SQL.



## Autograder Setup

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
import re
import os
import platform
if re.search(r'amzn', platform.uname().release): 
    GS = True
else: 
    GS = False
GS

False

In [3]:
if IN_COLAB == True: 
    print("Installing otter:")
    !pip install otter-grader==4.2.0 

Installing otter:
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 204 kB 5.0 MB/s 
     |████████████████████████████████| 100 kB 3.0 MB/s 
     |████████████████████████████████| 297 kB 63.3 MB/s 
     |████████████████████████████████| 1.6 MB 65.9 MB/s 
     |████████████████████████████████| 46 kB 4.3 MB/s 
     |████████████████████████████████| 84 kB 3.4 MB/s 


All the files for the project are available as downloads from Canvas or on Kaggle for the main database.  

Finally, let's get the rest of the imports setup.  

In [7]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline

import sqlite3

import getpass
import otter
grader = otter.Notebook()

# SQL 

For this portion of the project you will look at querying a database `fec.db` that is a portion of the information reported to the Federal Elections Committee.  

Details on the form of the full data is provided here: https://www.fec.gov/data/browse-data/?tab=bulk-data
  

## Table Descriptions 

Below is a list of the tables in the database.  Each table links to the documentation on the FEC page for the dataset. 

* [cn](https://www.fec.gov/campaign-finance-data/all-candidates-file-description/) -   Candidates table.  Contains id, names and party affiliation, ... 
* [cm](https://www.fec.gov/campaign-finance-data/committee-master-file-description/) -   Committee table.  Contains committee names and types ...
* [ccl](https://www.fec.gov/campaign-finance-data/candidate-committee-linkage-file-description/) -   Committee-Candidate Linkage table.  Contains information linking the candidate's information to information about his or her committee. 
* [oth](https://www.fec.gov/campaign-finance-data/any-transaction-one-committee-another-file-description/) -  Committe-Committee transaction table.  Contains each contribution or independent expenditure that one committee gives to another. 
* [pas2](https://www.fec.gov/campaign-finance-data/contributions-committees-candidates-file-description/) - Committee-Candidate Contributions table.  Contributions from committees to candidates file is a subset of the itemized records (OTH) file.
* [indiv](https://www.fec.gov/campaign-finance-data/contributions-individuals-file-description/) -   Individual contributions table.  Contains information for contributions given by individuals.  
The raw data has over 130,000,000 entries, you will look at a restricted list with contributions from Michigan to President committees between Oct. 14, 2019-Jan. 1, 2020.
* [pres](https://www.fec.gov/data/browse-data/?tab=candidates) - Presidential candidate table.  Financial information disclosed by presidential candidates who have reported at least $100,000 in contributions from individuals other than the candidate.
* [contByZip](https://www.fec.gov/data/browse-data/?tab=candidates) - Contribution totals by state and 3-digit zip code table. Presidential financial contributions by state and 3-digit zip codes. 



In [8]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


We will again make use of the functions `print_sql` or `pretty_print_sql` to look at the results of the SQL queries. 

In [9]:
def print_sql(c, s):
    print('>', s)
    for result in c.execute(s):
        print(result)
    print()

In [10]:
def pretty_print_sql(c, s):
    print('>', s)
    df = pd.read_sql(s, c)
    display(df)
    return df

## Access the Database 

The FEC data will again be made available to download from [Kaggle](https://www.kaggle.com).

Use your Kaggle credentials: `username` and `key` to access the data. 



Copy and paste the key information (does not upload your key information to the Drive). 

In [11]:
if GS==False: 
    import os
    from getpass import getpass
    os.environ['KAGGLE_USERNAME'] = getpass("Enter Kaggle Username: ")
    os.environ['KAGGLE_KEY'] = getpass("Enter Kaggle API Key: ")

Enter Kaggle Username: ··········
Enter Kaggle API Key: ··········


####  Download the Data 

In [12]:
if GS==False:
    !kaggle datasets download -d lebrown/un5550fec # api copied from kaggle
    !unzip un5550fec.zip

 94% 129M/137M [00:00<00:00, 180MB/s]
100% 137M/137M [00:00<00:00, 167MB/s]
Archive:  un5550fec.zip
  inflating: fec.db                  


You now have the `fec.db` database available to use. 

## Q1 - Connect to Database (5 pts)

Connect to `fec.db` and ensure you can access tables. 

For example, run the following query: `SELECT * from cn LIMIT 5`

In [13]:
# connect to database 
conn = sqlite3.connect("fec.db")

# Run first query
q1df = pretty_print_sql(conn, '''SELECT * from cn LIMIT 5''')


> SELECT * from cn LIMIT 5


,cand_id,cand_name,cand_pty_affiliation,cand_election_yr,cand_office_st,cand_office,cand_office_district,cand_ici,cand_status,cand_pcc,cand_st1,cand_st2,cand_city,cand_st,cand_zip
0,H0AK00105,"LAMB, THOMAS",NNE,2020,AK,H,0.0,C,N,C00607515,1861 W LAKE LUCILLE DR,None,WASILLA,AK,99654.0
1,H0AK00113,"TUGATUK, RAY SEAN",DEM,2020,AK,H,0.0,C,N,None,PO BOX 172,None,MANAKOTAK,AK,99628.0
2,H0AK01046,"CATALANO, THOMAS",OTH,2020,AK,H,0.0,None,N,None,188 WEST NORTHERN LIGHTS BOULEVARD,None,ANCHORAGE,AK,99503.0
3,H0AL01055,"CARL, JERRY LEE, JR",REP,2020,AL,H,1.0,O,C,C00697789,PO BOX 852138,None,MOBILE,AL,36685.0
4,H0AL01063,"LAMBERT, DOUGLAS WESTLEY III",REP,2020,AL,H,1.0,O,C,C00701557,7194 STILLWATER BLVD,None,SPANISH FORT,AL,36527.0


In [17]:
grader.check("q1")

q1 results: All test cases passed!

### Exploring a single table - `oth`

Take a look at the information stored in the `oth` table. 

In [18]:
pretty_print_sql(conn, """
SELECT *
FROM oth
LIMIT 25
""")

> 
SELECT *
FROM oth
LIMIT 25



,cmte_id,amndt_ind,rpt_tp,transaction_pgi,image_num,transaction_tp,entity_tp,name,city,state,...,employer,occupation,transaction_dt,transaction_amt,other_id,tran_id,file_num,memo_cd,memo_text,sub_id
0,C00567180,T,TER,P2020,201901099143774199,24K,PAC,TED YOHO FOR CONGRESS,GAINESVILLE,FL,...,None,None,1082019.0,1880,C00494583,SB23.4447,1303604,None,None,4021920191640423011
1,C00493916,T,TER,P,201902059145462342,20C,CAN,"BELLAVIA, DAVID GREGORY",BATAVIA,NY,...,None,None,2042019.0,6504,H8NY26087,SB19A.6174,1314408,None,None,4022020191640624767
2,C00634709,T,TER,P2017,201902059145462658,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,80000,H8SC05174,VTDDRAC1456,1314437,X,*,4021920191640447142
3,C00634709,T,TER,P2017,201902059145462658,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,125000,H8SC05174,VTDDRAC1464,1314437,X,*,4021920191640447144
4,C00634709,T,TER,P2018,201902059145462658,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,80000,H8SC05174,VTDDRAC1471,1314437,X,*,4021920191640447146
5,C00634709,T,TER,P2018,201902059145462659,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,20000,H8SC05174,VTDDRAC1489,1314437,X,*,4021920191640447148
6,C00634709,T,TER,P2018,201902059145462659,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,90000,H8SC05174,VTDDRAC1497,1314437,X,*,4021920191640447150
7,C00634709,T,TER,G2018,201902059145462659,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,10000,H8SC05174,VTDDRAC14A5,1314437,X,*,4021920191640447152
8,C00634709,T,TER,G2018,201902059145462660,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,2848,H8SC05174,VTDDRAC14B3,1314437,X,*,4021920191640447154
9,C00634709,T,TER,G2018,201902059145462660,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,12151,H8SC05174,VTDDRAC14C1,1314437,None,None,4021920191640447156


,cmte_id,amndt_ind,rpt_tp,transaction_pgi,image_num,transaction_tp,entity_tp,name,city,state,...,employer,occupation,transaction_dt,transaction_amt,other_id,tran_id,file_num,memo_cd,memo_text,sub_id
0,C00567180,T,TER,P2020,201901099143774199,24K,PAC,TED YOHO FOR CONGRESS,GAINESVILLE,FL,...,None,None,1082019.0,1880,C00494583,SB23.4447,1303604,None,None,4021920191640423011
1,C00493916,T,TER,P,201902059145462342,20C,CAN,"BELLAVIA, DAVID GREGORY",BATAVIA,NY,...,None,None,2042019.0,6504,H8NY26087,SB19A.6174,1314408,None,None,4022020191640624767
2,C00634709,T,TER,P2017,201902059145462658,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,80000,H8SC05174,VTDDRAC1456,1314437,X,*,4021920191640447142
3,C00634709,T,TER,P2017,201902059145462658,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,125000,H8SC05174,VTDDRAC1464,1314437,X,*,4021920191640447144
4,C00634709,T,TER,P2018,201902059145462658,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,80000,H8SC05174,VTDDRAC1471,1314437,X,*,4021920191640447146
5,C00634709,T,TER,P2018,201902059145462659,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,20000,H8SC05174,VTDDRAC1489,1314437,X,*,4021920191640447148
6,C00634709,T,TER,P2018,201902059145462659,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,90000,H8SC05174,VTDDRAC1497,1314437,X,*,4021920191640447150
7,C00634709,T,TER,G2018,201902059145462659,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,10000,H8SC05174,VTDDRAC14A5,1314437,X,*,4021920191640447152
8,C00634709,T,TER,G2018,201902059145462660,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,2848,H8SC05174,VTDDRAC14B3,1314437,X,*,4021920191640447154
9,C00634709,T,TER,G2018,201902059145462660,20C,CAN,"PARNELL, ARCHIE",SUMTER,SC,...,None,None,2042019.0,12151,H8SC05174,VTDDRAC14C1,1314437,None,None,4021920191640447156


You can use a similar query to explore the structure of the other tables:  `cn`, `cm`, `oth`, `pas2`, `ccl`, `indiv`, `pres`.

In [19]:
temp = pretty_print_sql(conn, """
SELECT *
FROM cn
LIMIT 5
""")

> 
SELECT *
FROM cn
LIMIT 5



,cand_id,cand_name,cand_pty_affiliation,cand_election_yr,cand_office_st,cand_office,cand_office_district,cand_ici,cand_status,cand_pcc,cand_st1,cand_st2,cand_city,cand_st,cand_zip
0,H0AK00105,"LAMB, THOMAS",NNE,2020,AK,H,0.0,C,N,C00607515,1861 W LAKE LUCILLE DR,None,WASILLA,AK,99654.0
1,H0AK00113,"TUGATUK, RAY SEAN",DEM,2020,AK,H,0.0,C,N,None,PO BOX 172,None,MANAKOTAK,AK,99628.0
2,H0AK01046,"CATALANO, THOMAS",OTH,2020,AK,H,0.0,None,N,None,188 WEST NORTHERN LIGHTS BOULEVARD,None,ANCHORAGE,AK,99503.0
3,H0AL01055,"CARL, JERRY LEE, JR",REP,2020,AL,H,1.0,O,C,C00697789,PO BOX 852138,None,MOBILE,AL,36685.0
4,H0AL01063,"LAMBERT, DOUGLAS WESTLEY III",REP,2020,AL,H,1.0,O,C,C00701557,7194 STILLWATER BLVD,None,SPANISH FORT,AL,36527.0


## Q2 - Number of PAC transactions (5 pts) 

We want to know the number of political actions committees (PACs) that donated large sums of money to the candidates. To begin to answer this question, we will look at the `oth` table. We'll find all the transactions that exceed $5,000.  Rather than print out the entire list, construct a query to count the number of entries. 

Make sure to only consider transactions involving a PAC. 

In [20]:
q2df = pretty_print_sql(conn, '''
SELECT COUNT(transaction_amt)
FROM oth
WHERE transaction_amt > 5000 AND
entity_tp = 'PAC'
''')


> 
SELECT COUNT(transaction_amt)
FROM oth
WHERE transaction_amt > 5000 AND
entity_tp = 'PAC'



,COUNT(transaction_amt)
0,9027


In [22]:
grader.check("q2")

q2 results: All test cases passed!

## Q3 - Transactions and PAC transactions >$5,000 (10 pts)

Having seen that there aren't too many transactions ($<$1 million) that exceed 5,000 dollars, let's find them all. 

Construct an SQL query to return a DataFrame containing the IDs of the two committees (`cmte_id` and `other_id`), and the transaction amount `trans_amt`, for transactions that exceed 5,000 dollars, limited to those involving PACs (`entity_tp`). 

Sort the results in decreasing order by transaction amount, and limit the return to 10 entries. 




In [23]:
q3df = pretty_print_sql(conn, '''
SELECT cmte_id, other_id, transaction_amt AS trans_amt
FROM oth
WHERE transaction_amt > 5000 AND
entity_tp = 'PAC'
ORDER BY transaction_amt DESC
LIMIT 10;
''')

> 
SELECT cmte_id, other_id, transaction_amt AS trans_amt
FROM oth
WHERE transaction_amt > 5000 AND
entity_tp = 'PAC'
ORDER BY transaction_amt DESC
LIMIT 10;



,cmte_id,other_id,trans_amt
0,C00693382,C00484642,5000000
1,C00484642,C00693382,5000000
2,C00701888,C00747246,3589500
3,C00747246,C00701888,3589500
4,C00737890,C00484642,3400000
5,C00484642,C00737890,3400000
6,C00484642,C00418897,3298000
7,C00486845,C00495861,3164400
8,C00693382,C00495861,3000000
9,C00490847,C00495028,3000000


In [24]:
grader.check("q3")

q3 results: All test cases passed!

## Q4 - Totals per committee (10 pts) 

We might see the same committee in this table multiple times. 

Therefore, let's report the total (sum of) transactions to a commitee ('cmte_id'). Return the committee ID, and total transaction amount ('total_trans_amt'). Limit your results to the top 10, sorted in descending order by amount.



In [25]:
q4df = pretty_print_sql(conn, '''
SELECT cmte_id, SUM(transaction_amt) AS total_trans_amt
FROM oth
GROUP BY cmte_id
ORDER BY total_trans_amt DESC
LIMIT 10;
''')

> 
SELECT cmte_id, SUM(transaction_amt) AS total_trans_amt
FROM oth
GROUP BY cmte_id
ORDER BY total_trans_amt DESC
LIMIT 10;



,cmte_id,total_trans_amt
0,C00003418,438843587
1,C00580100,377195445
2,C00010603,214684417
3,C00618371,160837965
4,C00075820,140897336
5,C00027466,115741793
6,C00703975,115055738
7,C00618389,113845539
8,C00695320,100006136
9,C00484642,89714928


In [26]:
grader.check("q4")

q4 results: All test cases passed!

# Queries combining tables

## Q5 - Contributions from committees to candidates (10 pts)

Let's focus on contributions from committees to candidates.  We want to see the total transactions (sum) between committees, `cmte_id` to candidates, `cand_id`. Note, this should only be new transactions `amndt_ind`.   

You will need to add additional information to the data from `pas2` by joining with the `cm` table and `cn` table in order to get the names of the committees and names of candidates involved in the transactions. 

Report the `cmte_id`, `cmte_name` (of the committee), `cand_id`, `cand_name`, and the `total_trans_amt`.  Sort by descending amounts and report the top 10.

In [27]:
q5df = pretty_print_sql(conn, '''
SELECT pas2.cmte_id, cm.cmte_nm AS cmte_name, pas2.cand_id,
cn.cand_name, SUM(pas2.transaction_amt) AS total_trans_amt
FROM pas2
LEFT JOIN cm 
ON pas2.cmte_id = cm.cmte_id
LEFT JOIN cn
ON pas2.cand_id = cn.cand_id
WHERE pas2.amndt_ind = 'N'
GROUP BY pas2.cmte_id, pas2.cand_id
ORDER BY total_trans_amt DESC
LIMIT 10
''')

> 
SELECT pas2.cmte_id, cm.cmte_nm AS cmte_name, pas2.cand_id,
cn.cand_name, SUM(pas2.transaction_amt) AS total_trans_amt
FROM pas2
LEFT JOIN cm 
ON pas2.cmte_id = cm.cmte_id
LEFT JOIN cn
ON pas2.cand_id = cn.cand_id
WHERE pas2.amndt_ind = 'N'
GROUP BY pas2.cmte_id, pas2.cand_id
ORDER BY total_trans_amt DESC
LIMIT 10



,cmte_id,cmte_name,cand_id,cand_name,total_trans_amt
0,C00637512,"AMERICA FIRST ACTION, INC.",P80000722,"BIDEN, JOSEPH R JR",37231810
1,C00495861,PRIORITIES USA ACTION,P80001571,"TRUMP, DONALD J.",24055100
2,C00725820,THE LINCOLN PROJECT,P80001571,"TRUMP, DONALD J.",20022102
3,C00495861,PRIORITIES USA ACTION,P80000722,"BIDEN, JOSEPH R JR",14189086
4,C00544767,THE COMMITTEE TO DEFEND THE PRESIDENT,P80001571,"TRUMP, DONALD J.",9936033
5,C00571588,RESTORATION PAC,P80000722,"BIDEN, JOSEPH R JR",8518540
6,C00486845,LCV VICTORY FUND,P80001571,"TRUMP, DONALD J.",8367217
7,C00608489,GREAT AMERICA PAC,P80001571,"TRUMP, DONALD J.",8005115
8,C00492140,AB PAC,P80001571,"TRUMP, DONALD J.",7677113
9,C00484642,SMP,S6ME00159,"COLLINS, SUSAN M.",6409313


In [28]:
grader.check("q5")

q5 results: All test cases passed!

## Q6 - Total/Counts of Contributions by party (10 pts)

For all the transactions in pas2, let's report the overall total transaction amount, and number of transactions for each party - again limited to new transactions (`amdnt_tp`). 

You will again need to add additional information to `pas2` by joining with the `cm` table and `cn` table in order to get the names of the committees and party of candidates involved in the transactions.

Report out candidate party (`cand_party`), total transaction (`total_trans_amt`), number of transactions (`cnt`), sorted by descending total amount.   Limit results to the top 10.

In [33]:
q6df = pretty_print_sql(conn, '''
SELECT cn.cand_pty_affiliation AS cand_party, 
SUM(pas2.transaction_amt) AS total_trans_amt,
COUNT(pas2.transaction_amt) AS cnt
FROM pas2
JOIN cm
ON pas2.cmte_id = cm.cmte_id
JOIN cn
ON pas2.cand_id = cn.cand_id
WHERE pas2.amndt_ind = 'N'
GROUP BY cn.cand_pty_affiliation
ORDER BY total_trans_amt DESC
LIMIT 10
''')

> 
SELECT cn.cand_pty_affiliation AS cand_party, 
SUM(pas2.transaction_amt) AS total_trans_amt,
COUNT(pas2.transaction_amt) AS cnt
FROM pas2
JOIN cm
ON pas2.cmte_id = cm.cmte_id
JOIN cn
ON pas2.cand_id = cn.cand_id
WHERE pas2.amndt_ind = 'N'
GROUP BY cn.cand_pty_affiliation
ORDER BY total_trans_amt DESC
LIMIT 10



,cand_party,total_trans_amt,cnt
0,REP,405175923,134540
1,DEM,378275551,95677
2,IND,2308036,236
3,DFL,1574886,524
4,NPP,91000,56
5,LIB,84980,44
6,GRE,27890,4
7,CRV,17000,4
8,NON,7162,19
9,OTH,1985,3


In [34]:
grader.check("q6")

q6 results: All test cases passed!

# Individual Contributions 

Let's now look at money coming from individual contributions - `indiv` table.   
Reminder the individual contributions are restricted to those from Michigan to President committees between Oct. 14, 2019-Jan. 1, 2020.

## Q7 - Individual Contributions (15 pts) 

Contributions are linked to candidate committee's not candidates directly. 

Compute the total contributions for each candidate's committee.  

Report out committee name (`cmte_name`), candidate name (`cand_name`), total amount (`tot_amt`), and number of contributions (`cnt`). Sorted by total amount limited to the top 10. 

You may need to join with the committee `cm` and president `pres` tables.


In [35]:
q7df = pretty_print_sql(conn, '''
SELECT indiv.committee_name AS cmte_name,
cn.cand_name, 
SUM(indiv.contribution_receipt_amount) AS tot_amt,
COUNT(indiv.contribution_receipt_amount) AS cnt
FROM indiv
LEFT JOIN pres
ON indiv.committee_id = pres.cmte_id
LEFT JOIN cm
ON indiv.committee_id = cm.cmte_id
LEFT JOIN cn
ON cm.cand_id = cn.cand_id
GROUP BY indiv.committee_name
ORDER BY SUM(indiv.contribution_receipt_amount) DESC
LIMIT 10
''')

> 
SELECT indiv.committee_name AS cmte_name,
cn.cand_name, 
SUM(indiv.contribution_receipt_amount) AS tot_amt,
COUNT(indiv.contribution_receipt_amount) AS cnt
FROM indiv
LEFT JOIN pres
ON indiv.committee_id = pres.cmte_id
LEFT JOIN cm
ON indiv.committee_id = cm.cmte_id
LEFT JOIN cn
ON cm.cand_id = cn.cand_id
GROUP BY indiv.committee_name
ORDER BY SUM(indiv.contribution_receipt_amount) DESC
LIMIT 10



,cmte_name,cand_name,tot_amt,cnt
0,"DONALD J. TRUMP FOR PRESIDENT, INC.","TRUMP, DONALD J.",10359650.60,154759
1,BIDEN FOR PRESIDENT,"BIDEN, JOSEPH R JR",7171871.33,77677
2,BERNIE 2020,"SANDERS, BERNARD",2047751.75,65598
3,"WARREN FOR PRESIDENT, INC.","WARREN, ELIZABETH",863940.02,16251
4,WIN THE ERA PAC,"BUTTIGIEG, PETE",713079.10,10358
5,AMY FOR AMERICA,"KLOBUCHAR, AMY J.",312441.39,3802
6,FRIENDS OF ANDREW YANG,"YANG, ANDREW MR.",312278.32,6533
7,KAMALA HARRIS FOR THE PEOPLE,"HARRIS, KAMALA D.",204622.94,2562
8,MARIANNE WILLIAMSON FOR PRESIDENT,"WILLIAMSON, MARIANNE",70993.69,1338
9,CORY 2020,"BOOKER, CORY A.",70949.17,1197


In [36]:
grader.check("q7")

q7 results: All test cases passed!

## Bonus1 - Individual Contributions (5 pts) 

We are interested in the proportion of money that comes from small donors - less than 200 dollars.  For example, if Biden raised 1000, with 300 from small donors, the proportion of small donors is 0.3.  

Compute this proportion for each candidate. The resulting table should have four columns: cmte_name, cand_name, total_amount, prop_small.  Limit results to top 10 candidates sorted by total_amount 

*Hint*: try using `CASE` statements to filter out transactions under $200. 

In [37]:
...
b1df = pretty_print_sql(conn, '''
WITH temp AS(
SELECT indiv.committee_name AS cmte_name,
cn.cand_name, 
SUM(indiv.contribution_receipt_amount) AS total_amount,
SUM(CASE
WHEN indiv.contribution_receipt_amount < 200 THEN indiv.contribution_receipt_amount
ELSE 0
END) AS small
FROM indiv
LEFT JOIN pres
ON indiv.committee_id = pres.cmte_id
LEFT JOIN cm
ON indiv.committee_id = cm.cmte_id
LEFT JOIN cn
ON cm.cand_id = cn.cand_id
GROUP BY indiv.committee_name)
SELECT cmte_name, cand_name, total_amount, small/total_amount AS prop_small
FROM temp
ORDER BY total_amount DESC
LIMIT 10
''')

> 
WITH temp AS(
SELECT indiv.committee_name AS cmte_name,
cn.cand_name, 
SUM(indiv.contribution_receipt_amount) AS total_amount,
SUM(CASE
WHEN indiv.contribution_receipt_amount < 200 THEN indiv.contribution_receipt_amount
ELSE 0
END) AS small
FROM indiv
LEFT JOIN pres
ON indiv.committee_id = pres.cmte_id
LEFT JOIN cm
ON indiv.committee_id = cm.cmte_id
LEFT JOIN cn
ON cm.cand_id = cn.cand_id
GROUP BY indiv.committee_name)
SELECT cmte_name, cand_name, total_amount, small/total_amount AS prop_small
FROM temp
ORDER BY total_amount DESC
LIMIT 10



,cmte_name,cand_name,total_amount,prop_small
0,"DONALD J. TRUMP FOR PRESIDENT, INC.","TRUMP, DONALD J.",10359650.60,0.528131
1,BIDEN FOR PRESIDENT,"BIDEN, JOSEPH R JR",7171871.33,0.332794
2,BERNIE 2020,"SANDERS, BERNARD",2047751.75,0.723780
3,"WARREN FOR PRESIDENT, INC.","WARREN, ELIZABETH",863940.02,0.617771
4,WIN THE ERA PAC,"BUTTIGIEG, PETE",713079.10,0.341989
5,AMY FOR AMERICA,"KLOBUCHAR, AMY J.",312441.39,0.379577
6,FRIENDS OF ANDREW YANG,"YANG, ANDREW MR.",312278.32,0.680150
7,KAMALA HARRIS FOR THE PEOPLE,"HARRIS, KAMALA D.",204622.94,0.341879
8,MARIANNE WILLIAMSON FOR PRESIDENT,"WILLIAMSON, MARIANNE",70993.69,0.615684
9,CORY 2020,"BOOKER, CORY A.",70949.17,0.522250


In [38]:
grader.check("b1")

b1 results: All test cases passed!

## Bonus2 (5 pts)

Let's look back at Q3 and adjust our query.  

Now, construct an SQL query to return a DataFrame containing the IDs of the two committees (`cmte_id` and `other_id`), and the number of transactions between them (`cnt`), for transactions that exceed 1,000 dollars, limited to those involving PACs (entity_tp).

Sort the items returned in decreasing order by `cnt`, and limit the return to 10 entries.

In [39]:
b2df = pretty_print_sql(conn, '''
SELECT cmte_id, other_id, COUNT(transaction_amt) AS cnt
FROM oth
WHERE transaction_amt > 1000 AND
entity_tp = 'PAC'
GROUP BY cmte_id, other_id
ORDER BY cnt DESC
LIMIT 10
''')

> 
SELECT cmte_id, other_id, COUNT(transaction_amt) AS cnt
FROM oth
WHERE transaction_amt > 1000 AND
entity_tp = 'PAC'
GROUP BY cmte_id, other_id
ORDER BY cnt DESC
LIMIT 10



,cmte_id,other_id,cnt
0,C00715029,C00715383,151
1,C00004036,C00462044,82
2,C00041160,C00694323,56
3,C00696948,C00401224,44
4,C00149641,C00010603,43
5,C00038505,C00003418,36
6,C00499392,C00715102,29
7,C00000935,C00492421,28
8,C00166710,C00042366,27
9,C00498121,C00583435,27


In [40]:
grader.check("b2")

b2 results: All test cases passed!

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

**NOTE** the following cell can not be run in Colab or Deepnote.  You can work in Colab and save the results, but the export function must be run in a Jupyter environment on the campus lab machines or your own computer.  See the instruction in the Canvas assignment.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False)